# README – Running this notebook

This notebook uses **RankingSHAP** as proposed by *Maria Heuss*, in the MQ2008 dataset.
We worked with her open-source code available on GitHub to reproduce and extend experiments.  
For a deeper understanding of the concept, please refer to the associated report where the methodology and interpretations are discussed in detail.

## Access the necessary files

All files required to run this notebook (even the dataset) are available in the following Drive folder:  
🔗 [RankingSHAP files on Drive](https://drive.google.com/drive/folders/1uoNotLMHnT8adxjhaTfmI2SKUKNDrxZb?usp=drive_link)

## How to run this notebook

There are two ways to set up the required files:

### 1 **Using Google Drive (recommended if you have a Drive account)**

- Download the `RankingShap` folder from the link above and place it into your own Google Drive.
- In the notebook, mount your Drive and navigate to the folder thanks to the two first celluls

###  2 Using local upload (if you prefer working locally)

- Download the folder from the Drive link above.

- Compress it as `RankingShap.zip` (if not already).

- Upload and unzip the folder in the notebook thanks to the third and fourth cellul


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/RankingShap

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!unzip -q RankingShap.zip

In [ ]:
"""
This cell installs all the required Python packages listed in the requirements.txt file.
"""

!pip install -r requirements.txt


## Dataset exploration

In these cells, we explore the distribution of the dataset.  
We compute and visualize the number of documents per query in the MQ2008 test set.  
A histogram shows the distribution, and summary statistics (total queries, average, min, max documents per query) are printed.


In [ ]:
"""
This cell defines a function to compute the number of documents per query ID (qid)
from a LETOR-style dataset file. It generates a histogram showing the distribution
of document counts per query for the test set, and prints summary statistics.
"""

from collections import Counter
import matplotlib.pyplot as plt

def count_docs_per_query(filepath):
    qid_counter = Counter()
    with open(filepath, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 2:
                continue  # Skip empty or invalid lines
            qid = int(parts[1].split(":")[1])  # Extract qid from the second field (format qid:XYZ)
            qid_counter[qid] += 1  # Count occurrence of each qid
    return qid_counter

# Load the test set file
file_test = "data/MQ2008/Fold1/test_1.txt"
combined_counts = count_docs_per_query(file_test)

# Get document counts per query for plotting
doc_per_query = list(combined_counts.values())

# Define histogram bins
bins = [0, 5, 10, 15, 20, 30, 50, 100]

# Plot histogram
plt.figure(figsize=(10, 6))
plt.hist(doc_per_query, bins=bins, color='skyblue', edgecolor='black')
plt.xlabel("Number of documents per query")
plt.ylabel("Number of queries")
plt.title("Distribution of documents per query in MQ2008 (test set)")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Print summary statistics
print(f"Total number of queries: {len(combined_counts)}")
print(f"Average number of documents per query: {sum(doc_per_query) / len(doc_per_query):.2f}")
print(f"Max documents in a query: {max(doc_per_query)}")
print(f"Min documents in a query: {min(doc_per_query)}")


## Query selection for analysis

In this study, we work on a subset of **50 queries** out of the original 136 queries in the MQ2008 test set.  

The selected queries represent the original dataset distribution in terms of the number of documents per query:  
- **10 rich queries** (with 30 or more documents)  
- **20 medium queries** (with between 11 and 29 documents)  
- **20 poor queries** (with fewer than 11 documents)  

This sampling ensures a balanced and representative selection for further analysis.


In [ ]:
rich = [qid for qid, c in combined_counts.items() if c >= 30]
medium = [qid for qid, c in combined_counts.items() if 11 <= c < 30]
poor = [qid for qid, c in combined_counts.items() if c < 11]

import random

random.seed(42)  # reproductability

selected_rich = random.sample(rich, min(10, len(rich)))
selected_medium = random.sample(medium, min(20, len(medium)))
selected_poor = random.sample(poor, min(20, len(poor)))

selected_queries = selected_rich + selected_medium + selected_poor
print(f"Selected {len(selected_queries)} queries")

import matplotlib.pyplot as plt

all_counts = [combined_counts[qid] for qid in selected_queries]
plt.hist(all_counts, bins=10, color='skyblue', edgecolor='black')
plt.xlabel("Number of documents per query (selected)")
plt.ylabel("Number of queries")
plt.title("Distribution of selected queries")
plt.show()

print(f"Rich: {len(selected_rich)} queries")
print(f"Medium: {len(selected_medium)} queries")
print(f"Poor: {len(selected_poor)} queries")
print(f"Total: {len(selected_queries)} queries")


## Test set filtering

We filtered the MQ2008 test set to create a new test file containing only the 50 selected queries.  
These queries represent different levels of document richness to ensure a balanced and representative subset.  

The steps performed were:
- Lines corresponding to the selected query IDs were extracted from the original `test_1.txt` file.
- The filtered data was saved as `test_selected50.txt`.
- The file was then renamed to `test.txt` to be used in further analysis.

This ensures that all subsequent processing is performed on the chosen subset of 50 queries.


In [ ]:
"""
This cell defines a function to read a LETOR-style dataset file and extract features, labels, and query IDs.
It loads the test set from 'test_1.txt', creates a DataFrame of qids, filters for the selected queries,
and displays the number of documents per selected query in descending order.
"""

import numpy as np
import pandas as pd

# Function to read a LETOR-style file
def read_letor_file(filepath):
    X = []
    y = []
    qids = []

    with open(filepath, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 2:
                continue  # Skip empty or invalid lines
            y_val = int(parts[0])
            qid = int(parts[1].split(":")[1])
            features = []
            for item in parts[2:]:
                if item.startswith("#"):
                    break  # Ignore comments
                _, val = item.split(":")
                features.append(float(val))
            X.append(features)
            y.append(y_val)
            qids.append(qid)

    return np.array(X), np.array(y), np.array(qids)

# Load the test set
X_test, y_test, qids_test = read_letor_file("data/MQ2008/Fold1/test_1.txt")

# Create a DataFrame for qids
df_qid = pd.DataFrame({'qid': qids_test})

# Filter for selected queries (assumes selected_queries already exists)
df_selected = df_qid[df_qid['qid'].isin(selected_queries)]

# Count documents per query and sort
query_doc_counts_selected = df_selected.groupby('qid').size().sort_values(ascending=False)

# Display results
print("✅ Selected queries and their document counts:")
print(query_doc_counts_selected.to_string())

# Optionally extract sorted query IDs
selected_query_ids = query_doc_counts_selected.index.tolist()
print("\nSelected query IDs (test set):", selected_query_ids)


In [ ]:
"""
This cell filters the test set file to include only the selected 50 queries,
writes the filtered data to a new file, and renames that file as the main test file for further use.
"""

input_path = "data/MQ2008/Fold1/test_1.txt"
output_path = "data/MQ2008/Fold1/test_selected50.txt"

# Set of selected qids as strings
selected_qids = {
    '19353', '19782', '18230', '19851', '18511', '18490', '18525', '18826', '19960', '18844',
    '19059', '18571', '19034', '18714', '18401', '19276', '19457', '18429', '19383', '19836',
    '18577', '18378', '19913', '18386', '18599', '18996', '19449', '18468', '19396', '19503',
    '19372', '19487', '19454', '19954', '19400', '19153', '19371', '19356', '19140', '19108',
    '18342', '19003', '18767', '18686', '18552', '18531', '19370', '18437', '18489', '18889'
}

# Read the input file and write lines that match the selected qids to the output file
with open(input_path, 'r') as infile, open(output_path, 'w') as outfile:
    for line in infile:
        qid_part = [p for p in line.split() if p.startswith("qid:")][0]
        qid = qid_part.split(":")[1]
        if qid in selected_qids:
            outfile.write(line)

print(f"✅ New file generated: {output_path}")

import os

# Rename the filtered file as the new test set
os.rename("data/MQ2008/Fold1/test_selected50.txt", "data/MQ2008/Fold1/test.txt")


## Model training and evaluation with RankingSHAP

In these steps, we train a listwise ranking model on the MQ2008 dataset and apply RankingSHAP for explanation.

- The `lime` package is installed to support model explainability.
- A ranking model is trained using `train_model.py` with the listwise approach.
- The RankingSHAP test script is made executable and run to generate explanations on the test set.

**Warning:** Running these cells will take a very long time (estimated runtime: up to 32 hours).  
Make sure you are prepared for extended computations before launching them.


In [ ]:
!pip install lime


In [ ]:
!python train_model.py --dataset MQ2008 --file_name test_model --model_type listwise


In [ ]:
!chmod +x run_rankingshap_test.sh

In [ ]:
!./run_rankingshap_test.sh

## Feature attribution visualization

These steps visualize the feature attributions computed by RankingSHAP:

- Individual bar plots are generated for each query, showing the attribution values for all features.
- A global bar plot is created by computing the mean attribution of each feature across all selected queries.  
  This provides an overall view of feature importance within the dataset.


In [ ]:
"""
This cell loads the RankingSHAP output file and generates bar plots for each query.
Each plot shows the feature attribution values computed by RankingSHAP for a given query.
"""

import pandas as pd
import matplotlib.pyplot as plt

# Load the RankingSHAP result file
df = pd.read_csv("results/results_MQ2008/feature_attributes/rankingshap.csv")

# List of query IDs from the CSV columns
query_ids = df.columns.tolist()
query_ids.remove("feature_number")  # Remove the feature number column

# Generate a bar plot for each query
for qid in query_ids:
    df_sorted = df.sort_values(by=qid, ascending=False)
    plt.figure(figsize=(10, 6))
    plt.bar(df_sorted['feature_number'].astype(str), df_sorted[qid])
    plt.title(f"RankingSHAP: Attribution of features for the query {qid}")
    plt.xlabel("Feature number")
    plt.ylabel("Attribution value")
    plt.xticks(rotation=45)
    plt.show()


In [ ]:
"""
This cell loads the RankingSHAP output file and computes the mean attribution value
for each feature across all selected queries. It generates a bar plot showing
the global importance of features based on these mean values.
"""

import pandas as pd
import matplotlib.pyplot as plt

# Load the RankingSHAP attribution file
df = pd.read_csv("results/results_MQ2008/feature_attributes/rankingshap.csv")

# Get the columns corresponding to query IDs
query_cols = [col for col in df.columns if col != "feature_number"]

# Compute the mean attribution value for each feature
df['mean_attribution'] = df[query_cols].mean(axis=1)

# Sort features by mean attribution in descending order
df_sorted = df.sort_values(by='mean_attribution', ascending=False)

# Plot the bar chart
plt.figure(figsize=(10, 6))
plt.bar(df_sorted['feature_number'].astype(str), df_sorted['mean_attribution'])
plt.title("RankingSHAP: Global Attribution of features (average on 50 queries)")
plt.xlabel("Feature number")
plt.ylabel("Mean attribution value")
plt.xticks(rotation=45)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Charger ton fichier RankingSHAP
df = pd.read_csv("results/results_MQ2008/feature_attributes/rankingshap.csv")

# Mapping of features
feature_map = {
    1: "TF-IDF body",
    2: "TF-IDF anchor",
    3: "TF-IDF title",
    4: "TF-IDF URL",
    5: "BM25 body",
    6: "BM25 anchor",
    7: "BM25 title",
    8: "BM25 URL",
    9: "LMIR.ABS body",
    10: "LMIR.ABS anchor",
    11: "LMIR.ABS title",
    12: "LMIR.ABS URL",
    13: "LMIR.DIR body",
    14: "LMIR.DIR anchor",
    15: "LMIR.DIR title",
    16: "LMIR.DIR URL",
    17: "LMIR.JM body",
    18: "LMIR.JM anchor",
    19: "LMIR.JM title",
    20: "LMIR.JM URL",
    21: "PageRank",
    22: "Inlink number",
    23: "Outlink number",
    24: "URL slashes count",
    25: "URL length",
    26: "Child pages count",
    27: "Query IDF",
    28: "Sum TF body",
    29: "Sum TF anchor",
    30: "Sum TF title",
    31: "Sum TF URL",
    32: "Min TF body",
    33: "Min TF anchor",
    34: "Min TF title",
    35: "Min TF URL",
    36: "Max TF body",
    37: "Max TF anchor",
    38: "Max TF title",
    39: "Max TF URL",
    40: "Avg TF body",
    41: "Avg TF anchor",
    42: "Avg TF title",
    43: "Avg TF URL",
    44: "Body length",
    45: "Anchor length",
    46: "Title length"
}

query_ids = df.columns.tolist()
query_ids.remove("feature_number")

df["feature_name"] = df["feature_number"].map(feature_map)

for qid in query_ids:
    df_sorted = df.sort_values(by=qid, ascending=True)  # ascending=True p
    plt.figure(figsize=(12, 10))
    plt.barh(df_sorted["feature_name"], df_sorted[qid])
    plt.title(f"RankingSHAP: Feature Attribution for query {qid}")
    plt.xlabel("Attribution value")
    plt.ylabel("Feature name")
    plt.tight_layout()
    plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Mapping of features
feature_map = {
    1: "TF-IDF body",
    2: "TF-IDF anchor",
    3: "TF-IDF title",
    4: "TF-IDF URL",
    5: "BM25 body",
    6: "BM25 anchor",
    7: "BM25 title",
    8: "BM25 URL",
    9: "LMIR.ABS body",
    10: "LMIR.ABS anchor",
    11: "LMIR.ABS title",
    12: "LMIR.ABS URL",
    13: "LMIR.DIR body",
    14: "LMIR.DIR anchor",
    15: "LMIR.DIR title",
    16: "LMIR.DIR URL",
    17: "LMIR.JM body",
    18: "LMIR.JM anchor",
    19: "LMIR.JM title",
    20: "LMIR.JM URL",
    21: "PageRank",
    22: "Inlink number",
    23: "Outlink number",
    24: "URL slashes count",
    25: "URL length",
    26: "Child pages count",
    27: "Query IDF",
    28: "Sum TF body",
    29: "Sum TF anchor",
    30: "Sum TF title",
    31: "Sum TF URL",
    32: "Min TF body",
    33: "Min TF anchor",
    34: "Min TF title",
    35: "Min TF URL",
    36: "Max TF body",
    37: "Max TF anchor",
    38: "Max TF title",
    39: "Max TF URL",
    40: "Avg TF body",
    41: "Avg TF anchor",
    42: "Avg TF title",
    43: "Avg TF URL",
    44: "Body length",
    45: "Anchor length",
    46: "Title length"
}

df = pd.read_csv("results/results_MQ2008/feature_attributes/rankingshap.csv")

query_cols = [col for col in df.columns if col != "feature_number"]

df['mean_attribution'] = df[query_cols].mean(axis=1)

df['feature_name'] = df['feature_number'].map(feature_map)

df_sorted = df.sort_values(by='mean_attribution', ascending=True)  # ascending=True

plt.figure(figsize=(12,10))
plt.barh(df_sorted['feature_name'], df_sorted['mean_attribution'])
plt.title("RankingSHAP: Global Feature Attribution (mean over 50 queries)")
plt.xlabel("Mean attribution value")
plt.ylabel("Feature name")
plt.tight_layout()
plt.show()


## Validation of top-k features through retraining and comparison

We validate whether the top-k features identified using the global RankingSHAP aggregation
are sufficient to maintain the model’s performance when used alone.  

The steps are as follows:
1️ Retrain the ranking model using only the top-k features selected based on their global importance.  
2️ Evaluate the performance of this model and compare it to the baseline.  
3️ Retrain another model using all features (full model).  
4️ Compare the performance of this full model to that of the top-k feature model
to demonstrate that the global feature ranking captures genuinely useful information for the model.


In [ ]:
"""
This cell computes the global mean attribution value for each feature
using RankingSHAP results. It generates:
- The list of top-k most important features based on mean attribution.
- Several sets of k randomly selected features for comparison.
"""

import pandas as pd
import numpy as np
import random

# Load the RankingSHAP attribution file
df = pd.read_csv("results/results_MQ2008/feature_attributes/rankingshap.csv")

# Get the columns corresponding to query IDs
query_cols = [col for col in df.columns if col != "feature_number"]

# Compute the mean attribution value for each feature
df['mean_attribution'] = df[query_cols].mean(axis=1)

# Sort features by mean attribution in descending order
df_sorted = df.sort_values(by='mean_attribution', ascending=False).reset_index(drop=True)

# Generate list of sorted feature names (e.g., 'feature_39', 'feature_30', ...)
sorted_feature_names = [f"feature_{int(f)}" for f in df_sorted['feature_number']]

# Parameters
k = 10  # you can change to k=20 if desired
n_random_repeats = 5

# Top-k features
top_k_features = sorted_feature_names[:k]

# Generate random-k feature sets
random_k_features_list = [
    random.sample(sorted_feature_names, k) for _ in range(n_random_repeats)
]

# Display results
print(f"\n=== Top-{k} Features ===")
print(top_k_features)

print(f"\n=== Random-{k} Features (repeats={n_random_repeats}) ===")
for i, feats in enumerate(random_k_features_list):
    print(f"Random set {i+1}: {feats}")


In [ ]:
"""
This cell:
- Loads the training, validation, and test datasets in LETOR format.
- Loads the RankingSHAP global feature ranking.
- Selects the top-k features based on mean attribution.
- Generates several sets of k random features for comparison.
- Filters the datasets to keep only the selected features (top-k and random-k).
- Saves these filtered datasets as CSV files for future experiments.
"""

import numpy as np
import pandas as pd
import random
random.seed(42)

# Function to read a LETOR-style file
def read_letor_file(filepath):
    X, y, qids = [], [], []
    with open(filepath, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 2:
                continue  # Skip invalid lines
            y_val = int(parts[0])
            qid = int(parts[1].split(":")[1])
            features = []
            for item in parts[2:]:
                if item.startswith("#"):
                    break  # Ignore comments
                _, val = item.split(":")
                features.append(float(val))
            X.append(features)
            y.append(y_val)
            qids.append(qid)
    return np.array(X), np.array(y), np.array(qids)

# Load train+valid and test sets
X_train, y_train, qids_train = read_letor_file("data/MQ2008/Fold1/train.txt")
X_test, y_test, qids_test = read_letor_file("data/MQ2008/Fold1/test_1.txt")

# Create DataFrames
feature_names = [f"feature_{i+1}" for i in range(X_train.shape[1])]
train_df = pd.DataFrame(X_train, columns=feature_names)
train_df['label'] = y_train
train_df['qid'] = qids_train

test_df = pd.DataFrame(X_test, columns=feature_names)
test_df['label'] = y_test
test_df['qid'] = qids_test

# Load feature ranking from RankingSHAP
df_rank = pd.read_csv("results/results_MQ2008/feature_attributes/rankingshap.csv")
query_cols = [col for col in df_rank.columns if col != "feature_number"]
df_rank['mean_attribution'] = df_rank[query_cols].mean(axis=1)
df_rank_sorted = df_rank.sort_values(by='mean_attribution', ascending=False).reset_index(drop=True)
sorted_feature_names = [f"feature_{int(f)}" for f in df_rank_sorted['feature_number']]

# Parameters
k = 10
n_random_repeats = 5

# Select top-k features
top_k_features = sorted_feature_names[:k]

# Generate random-k feature sets
random_k_features_list = [
    random.sample(feature_names, k) for _ in range(n_random_repeats)
]

# Function to filter dataset columns
def filter_dataset(df, feature_list):
    cols_to_keep = ['label', 'qid'] + feature_list
    return df[cols_to_keep].copy()

# Create filtered datasets
train_top_k = filter_dataset(train_df, top_k_features)
test_top_k = filter_dataset(test_df, top_k_features)

train_random_k_list = [filter_dataset(train_df, feats) for feats in random_k_features_list]
test_random_k_list = [filter_dataset(test_df, feats) for feats in random_k_features_list]

# Display selected features
print(f"✅ Top-{k} features selected:")
print(top_k_features)

for i, feats in enumerate(random_k_features_list):
    print(f"Random-{k} set {i+1}: {feats}")

# Save datasets to CSV files
train_top_k.to_csv("train_top_k.csv", index=False)
test_top_k.to_csv("test_top_k.csv", index=False)
for i, (train_r, test_r) in enumerate(zip(train_random_k_list, test_random_k_list)):
    train_r.to_csv(f"train_random_k_{i+1}.csv", index=False)
    test_r.to_csv(f"test_random_k_{i+1}.csv", index=False)


In [ ]:
"""
This cell:
- Defines parameters for fair comparison of models.
- Implements a function to train a LambdaMART LightGBM model.
- Trains a model using the top-k selected features.
- Trains multiple models using random-k selected features.
"""

import lightgbm as lgb
import numpy as np

# General parameters (fixed for fair comparison)
num_leaves = 31
n_estimators = 100
learning_rate = 0.1
random_seed = 42

# Function to train a LambdaMART LightGBM model
def train_lgb_ranker(train_df):
    # Separate features, labels, and query groups
    X_train = train_df.drop(columns=['label', 'qid'])
    y_train = train_df['label']
    group_train = train_df.groupby('qid').size().to_numpy()

    # Create LightGBM dataset
    lgb_train = lgb.Dataset(X_train, label=y_train, group=group_train)

    # Train the model
    model = lgb.train(
        {
            'objective': 'lambdarank',
            'metric': 'ndcg',
            'learning_rate': learning_rate,
            'num_leaves': num_leaves,
            'verbose': -1,
            'seed': random_seed
        },
        lgb_train,
        num_boost_round=n_estimators
    )
    return model

# Train the Top-k model
print("🚀 Training Top-k model...")
model_top_k = train_lgb_ranker(train_top_k)

# Train the Random-k models
model_random_k_list = []
for i, train_r in enumerate(train_random_k_list):
    print(f"🚀 Training Random-k model {i+1}...")
    model_r = train_lgb_ranker(train_r)
    model_random_k_list.append(model_r)


In [ ]:
"""
This cell:
- Defines a function to generate predictions grouped by query ID (qid).
- Defines a function to compute ranking metrics: NDCG@10, MAP, and Precision@10.
- Evaluates these metrics on the model trained with top-k features.
- Evaluates these metrics on models trained with random-k features.
"""

from sklearn.metrics import average_precision_score
import numpy as np
import warnings

# Function to generate predictions grouped by qid
def predict_and_group(model, test_df):
    X_test = test_df.drop(columns=['label', 'qid'])
    y_test = test_df['label']
    qid_test = test_df['qid']
    y_pred = model.predict(X_test)

    grouped = []
    for qid in np.unique(qid_test):
        mask = qid_test == qid
        grouped.append({
            'qid': qid,
            'y_true': y_test[mask].to_numpy(),
            'y_pred': y_pred[mask]
        })
    return grouped

# Function to compute ranking metrics
def eval_ranking(grouped_data, k=10):
    ndcg_list = []
    map_list = []
    prec_list = []

    for group in grouped_data:
        y_true = np.ravel(group['y_true'])
        y_pred = np.ravel(group['y_pred'])

        if len(y_true) < 2:
            continue  # Skip groups with insufficient documents

        order = np.argsort(-y_pred)
        y_true_sorted = y_true[order]

        # NDCG@k
        dcg = np.sum((2 ** y_true_sorted[:k] - 1) / np.log2(np.arange(2, 2 + len(y_true_sorted[:k]))))
        ideal_order = np.sort(y_true)[::-1]
        ideal_dcg = np.sum((2 ** ideal_order[:k] - 1) / np.log2(np.arange(2, 2 + len(ideal_order[:k]))))
        ndcg = dcg / ideal_dcg if ideal_dcg > 0 else 0.0
        ndcg_list.append(ndcg)

        # MAP
        try:
            if np.sum(y_true) > 0:
                ap = average_precision_score(y_true, y_pred)
            else:
                ap = 0.0
        except ValueError:
            ap = 0.0
        map_list.append(ap)

        # Precision@k
        prec = np.sum(y_true_sorted[:k]) / k
        prec_list.append(prec)

    return {
        'NDCG@10': np.mean(ndcg_list) if ndcg_list else 0.0,
        'MAP': np.mean(map_list) if map_list else 0.0,
        'Precision@10': np.mean(prec_list) if prec_list else 0.0
    }

# Evaluate Top-k model
print("✅ Evaluating Top-k model...")
grouped_top = predict_and_group(model_top_k, test_top_k)
results_top = eval_ranking(grouped_top)
print("Top-k results:", results_top)

# Evaluate Random-k models
results_random_list = []
for i, (model_r, test_r) in enumerate(zip(model_random_k_list, test_random_k_list)):
    print(f"✅ Evaluating Random-k model {i+1}...")
    grouped_r = predict_and_group(model_r, test_r)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=UserWarning)
        results_r = eval_ranking(grouped_r)
    results_random_list.append(results_r)
    print(f"Random-k {i+1} results:", results_r)


In [ ]:
"""
This cell:
- Defines a function to plot a bar chart comparing NDCG@10 for the top-k model and the mean of random-k models, including error bars for the random-k standard deviation.
- Extracts NDCG@10, MAP, and Precision@10 values from the random-k evaluation results.
- Computes summary statistics (mean and standard deviation) for these metrics.
- Visualizes the NDCG@10 comparison in a bar chart.
"""

import matplotlib.pyplot as plt

def plot_bar_ndcg(top_val, random_mean, random_std):
    labels = ['Top-k', 'Random-k mean']
    ndcg_values = [top_val, random_mean]
    errors = [0, random_std]  # No error bar for Top-k

    fig, ax = plt.subplots(figsize=(6, 4))
    ax.bar(labels, ndcg_values, yerr=errors, capsize=5, color=['tab:blue', 'tab:orange'])
    ax.set_ylabel('NDCG@10')
    ax.set_title('NDCG@10: Top-k vs Random-k')
    ax.set_ylim(0, 1.0)
    plt.tight_layout()
    plt.show()

# Extract NDCG@10, MAP, and Precision@10 values from random-k results
ndcg_vals = [r['NDCG@10'] for r in results_random_list]
map_vals = [r['MAP'] for r in results_random_list]
prec_vals = [r['Precision@10'] for r in results_random_list]

# Compute summary statistics
summary_random = {
    'NDCG@10 mean': np.mean(ndcg_vals),
    'NDCG@10 std': np.std(ndcg_vals),
    'MAP mean': np.mean(map_vals),
    'MAP std': np.std(map_vals),
    'Precision@10 mean': np.mean(prec_vals),
    'Precision@10 std': np.std(prec_vals)
}

# Visualize NDCG@10 comparison
plot_bar_ndcg(results_top['NDCG@10'], summary_random['NDCG@10 mean'], summary_random['NDCG@10 std'])


In [ ]:
"""
This cell:
- Defines a generic plotting function to compare Top-k and Random-k models for any ranking metric.
- Generates bar charts for MAP and Precision@10, showing the Top-k value and the mean ± standard deviation of Random-k.
"""

import matplotlib.pyplot as plt

def plot_bar_metric(metric_name, top_val, random_mean, random_std):
    labels = ['Top-k', 'Random-k mean']
    values = [top_val, random_mean]
    errors = [0, random_std]

    fig, ax = plt.subplots(figsize=(6, 4))
    ax.bar(labels, values, yerr=errors, capsize=5, color=['tab:green', 'tab:purple'])
    ax.set_ylabel(metric_name)
    ax.set_title(f'{metric_name}: Top-k vs Random-k')
    ax.set_ylim(0, 1.0)
    plt.tight_layout()
    plt.show()

# Generate plots for MAP and Precision@10
plot_bar_metric('MAP', results_top['MAP'], summary_random['MAP mean'], summary_random['MAP std'])
plot_bar_metric('Precision@10', results_top['Precision@10'], summary_random['Precision@10 mean'], summary_random['Precision@10 std'])


In [ ]:
"""
This cell:
- Defines a function to compare the performance of the Top-k model to the full-feature model for a given metric.
- Trains a model using all features (full-feature model).
- Evaluates the full-feature model on the full test set.
- Computes and prints the percentage of full performance retained by the Top-k model for NDCG@10, MAP, and Precision@10.
"""

def compare_to_full(top_result, full_result, metric='NDCG@10'):
    if full_result > 0:
        perc = 100 * (top_result / full_result)
    else:
        perc = 0.0
    print(f"✅ Top-k retains {perc:.2f}% of full {metric} performance (Top-k: {top_result:.4f}, Full: {full_result:.4f})")
    return perc

# Train the full-feature model
model_full = train_lgb_ranker(train_df)  # train_df: full training dataset with all features

# Evaluate the full-feature model on the full test set
grouped_full = predict_and_group(model_full, test_df)  # test_df: full test dataset with all features
results_full = eval_ranking(grouped_full)

# Compare Top-k performance to full-feature model
perc_ndcg = compare_to_full(results_top['NDCG@10'], results_full['NDCG@10'], 'NDCG@10')
perc_map = compare_to_full(results_top['MAP'], results_full['MAP'], 'MAP')
perc_prec = compare_to_full(results_top['Precision@10'], results_full['Precision@10'], 'Precision@10')


In [ ]:
def compare_top_random(top_val, random_mean, random_std, metric):
    diff = top_val - random_mean
    print(f"✅ Top-k {metric}: {top_val:.4f} | Random-k mean: {random_mean:.4f} ± {random_std:.4f} | Diff: {diff:.4f}")
    return diff

# Compare NDCG@10
diff_ndcg = compare_top_random(
    results_top['NDCG@10'],
    summary_random['NDCG@10 mean'],
    summary_random['NDCG@10 std'],
    'NDCG@10'
)

# Compare MAP
diff_map = compare_top_random(
    results_top['MAP'],
    summary_random['MAP mean'],
    summary_random['MAP std'],
    'MAP'
)

# Compare Precision@10
diff_prec = compare_top_random(
    results_top['Precision@10'],
    summary_random['Precision@10 mean'],
    summary_random['Precision@10 std'],
    'Precision@10'
)


## Validation of top-k features with varying k

We now repeat the same experiments while varying the value of k.  
This allows us to assess how the number of top features impacts model performance,  
and whether a larger or smaller subset of features provides a better balance between simplicity and effectiveness.

The process includes:
- Retraining models using top-k features for different values of k.
- Comparing their performance to models using k randomly selected features.
- Evaluating how much of the full model’s performance is retained as k changes.


In [ ]:
"""
This cell:
- Loads training and test data in LETOR format.
- Loads the global RankingSHAP feature ranking.
- Iterates over different values of k (from 5 to 20).
- For each k:
    - Selects the top-k features based on global RankingSHAP attribution.
    - Generates multiple random-k feature sets for comparison.
    - Trains a LightGBM LambdaMART model on top-k features.
    - Trains LightGBM models on each random-k feature set.
    - Saves the corresponding datasets for future analysis.
"""

import numpy as np
import pandas as pd
import random
import lightgbm as lgb

random.seed(42)

# Function to read a LETOR-style file
def read_letor_file(filepath):
    X, y, qids = [], [], []
    with open(filepath, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 2:
                continue  # Skip invalid lines
            y_val = int(parts[0])
            qid = int(parts[1].split(":")[1])
            features = []
            for item in parts[2:]:
                if item.startswith("#"):
                    break  # Ignore comments
                _, val = item.split(":")
                features.append(float(val))
            X.append(features)
            y.append(y_val)
            qids.append(qid)
    return np.array(X), np.array(y), np.array(qids)

# Load train and test sets
X_train, y_train, qids_train = read_letor_file("data/MQ2008/Fold1/train.txt")
X_test, y_test, qids_test = read_letor_file("data/MQ2008/Fold1/test_1.txt")

feature_names = [f"feature_{i+1}" for i in range(X_train.shape[1])]

train_df = pd.DataFrame(X_train, columns=feature_names)
train_df['label'] = y_train
train_df['qid'] = qids_train

test_df = pd.DataFrame(X_test, columns=feature_names)
test_df['label'] = y_test
test_df['qid'] = qids_test

# Load feature ranking
df_rank = pd.read_csv("results/results_MQ2008/feature_attributes/rankingshap.csv")
query_cols = [col for col in df_rank.columns if col != "feature_number"]
df_rank['mean_attribution'] = df_rank[query_cols].mean(axis=1)
df_rank_sorted = df_rank.sort_values(by='mean_attribution', ascending=False).reset_index(drop=True)
sorted_feature_names = [f"feature_{int(f)}" for f in df_rank_sorted['feature_number']]

# Function to filter dataset columns
def filter_dataset(df, feature_list):
    cols_to_keep = ['label', 'qid'] + feature_list
    return df[cols_to_keep].copy()

# Function to train a LightGBM LambdaMART model
def train_lgb_ranker(train_df):
    X_train = train_df.drop(columns=['label', 'qid'])
    y_train = train_df['label']
    group_train = train_df.groupby('qid').size().to_numpy()
    lgb_train = lgb.Dataset(X_train, label=y_train, group=group_train)
    model = lgb.train(
        {
            'objective': 'lambdarank',
            'metric': 'ndcg',
            'learning_rate': 0.1,
            'num_leaves': 31,
            'verbose': -1,
            'seed': 42
        },
        lgb_train,
        num_boost_round=100
    )
    return model

# Iterate over k values
for k in range(5, 21):
    print(f"\n🔹🔹 Processing k={k} 🔹🔹")

    top_k_features = sorted_feature_names[:k]
    random_k_features_list = [random.sample(feature_names, k) for _ in range(5)]

    # Top-k
    train_top_k = filter_dataset(train_df, top_k_features)
    test_top_k = filter_dataset(test_df, top_k_features)
    print(f"✅ Top-{k} features: {top_k_features}")
    model_top_k = train_lgb_ranker(train_top_k)

    # Save datasets if needed
    train_top_k.to_csv(f"train_top_k_{k}.csv", index=False)
    test_top_k.to_csv(f"test_top_k_{k}.csv", index=False)

    # Random-k
    model_random_k_list = []
    for i, feats in enumerate(random_k_features_list):
        print(f"   🚀 Training Random-k {i+1}: {feats}")
        train_r = filter_dataset(train_df, feats)
        test_r = filter_dataset(test_df, feats)
        model_r = train_lgb_ranker(train_r)
        model_random_k_list.append(model_r)

        # Save datasets if needed
        train_r.to_csv(f"train_random_k{k}_{i+1}.csv", index=False)
        test_r.to_csv(f"test_random_k{k}_{i+1}.csv", index=False)


In [ ]:
"""
This cell:
- Trains LightGBM LambdaMART models for different values of k (from 5 to 20) using both top-k and random-k feature sets.
- Evaluates each model on the corresponding test set.
- Computes NDCG@10, MAP, and Precision@10 for both top-k and random-k models.
- Aggregates and summarizes the results across all k values.
- Saves the results as a CSV file for further analysis.
"""

from sklearn.metrics import average_precision_score
import numpy as np
import warnings
import lightgbm as lgb

# Dictionaries to store models and test sets
models_top_k = {}
tests_top_k = {}
models_random_k = {}
tests_random_k = {}

for k in range(5, 21):
    print(f"\n⚡ Preparing k={k}")

    # Top-k features
    top_k_features = sorted_feature_names[:k]
    train_top = filter_dataset(train_df, top_k_features)
    test_top = filter_dataset(test_df, top_k_features)

    model_top = train_lgb_ranker(train_top)

    models_top_k[k] = model_top
    tests_top_k[k] = test_top

    # Random-k features
    random_models = []
    random_tests = []
    random_k_features_list = [random.sample(feature_names, k) for _ in range(5)]

    for feats in random_k_features_list:
        train_rand = filter_dataset(train_df, feats)
        test_rand = filter_dataset(test_df, feats)

        model_rand = train_lgb_ranker(train_rand)

        random_models.append(model_rand)
        random_tests.append(test_rand)

    models_random_k[k] = random_models
    tests_random_k[k] = random_tests

# Functions for prediction and evaluation
def predict_and_group(model, test_df):
    X_test = test_df.drop(columns=['label', 'qid'])
    y_test = test_df['label']
    qid_test = test_df['qid']
    y_pred = model.predict(X_test)

    grouped = []
    for qid in np.unique(qid_test):
        mask = qid_test == qid
        grouped.append({
            'qid': qid,
            'y_true': y_test[mask].to_numpy(),
            'y_pred': y_pred[mask]
        })
    return grouped

def eval_ranking(grouped_data, k=10):
    ndcg_list = []
    map_list = []
    prec_list = []

    for group in grouped_data:
        y_true = np.ravel(group['y_true'])
        y_pred = np.ravel(group['y_pred'])

        if len(y_true) < 2:
            continue

        order = np.argsort(-y_pred)
        y_true_sorted = y_true[order]

        # NDCG@k
        dcg = np.sum((2 ** y_true_sorted[:k] - 1) / np.log2(np.arange(2, 2 + len(y_true_sorted[:k]))))
        ideal_order = np.sort(y_true)[::-1]
        ideal_dcg = np.sum((2 ** ideal_order[:k] - 1) / np.log2(np.arange(2, 2 + len(ideal_order[:k]))))
        ndcg = dcg / ideal_dcg if ideal_dcg > 0 else 0.0
        ndcg_list.append(ndcg)

        # MAP
        try:
            if np.sum(y_true) > 0:
                ap = average_precision_score(y_true, y_pred)
            else:
                ap = 0.0
        except ValueError:
            ap = 0.0
        map_list.append(ap)

        # Precision@k
        prec = np.sum(y_true_sorted[:k]) / k
        prec_list.append(prec)

    return {
        'NDCG@10': np.mean(ndcg_list) if ndcg_list else 0.0,
        'MAP': np.mean(map_list) if map_list else 0.0,
        'Precision@10': np.mean(prec_list) if prec_list else 0.0
    }

# Evaluate and collect results
all_results = []

for k in range(5, 21):
    print(f"\n📊 Evaluating k={k} Top-k model...")
    grouped_top = predict_and_group(models_top_k[k], tests_top_k[k])
    results_top = eval_ranking(grouped_top)
    print(f"Top-{k} results: {results_top}")

    k_result = {
        'k': k,
        'Top NDCG@10': results_top['NDCG@10'],
        'Top MAP': results_top['MAP'],
        'Top Precision@10': results_top['Precision@10']
    }

    ndcg_random = []
    map_random = []
    prec_random = []

    for i, (model_r, test_r) in enumerate(zip(models_random_k[k], tests_random_k[k])):
        print(f"   ✅ Evaluating Random-k {i+1} model...")
        grouped_r = predict_and_group(model_r, test_r)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=UserWarning)
            results_r = eval_ranking(grouped_r)
        print(f"   Random-{k} {i+1} results: {results_r}")

        ndcg_random.append(results_r['NDCG@10'])
        map_random.append(results_r['MAP'])
        prec_random.append(results_r['Precision@10'])

    k_result.update({
        'Random NDCG@10 mean': np.mean(ndcg_random),
        'Random NDCG@10 std': np.std(ndcg_random),
        'Random MAP mean': np.mean(map_random),
        'Random MAP std': np.std(map_random),
        'Random Precision@10 mean': np.mean(prec_random),
        'Random Precision@10 std': np.std(prec_random)
    })

    all_results.append(k_result)

# Summarize and save results
df_results = pd.DataFrame(all_results)
print("\n📈 Overall results:")
print(df_results)

df_results.to_csv("ranking_summary_all_k.csv", index=False)


In [ ]:
"""
This cell:
- Trains a full-feature LightGBM LambdaMART model once.
- Evaluates the full-feature model on the full test set.
- Defines a function to compare Top-k model performance to the full-feature model for each metric.
- Updates the results to include the percentage of full model performance retained by each Top-k model.
- Saves the final summary as a CSV file.
"""

# Train the full-feature model once
print("🚀 Training full feature model...")
model_full = train_lgb_ranker(train_df)

# Evaluate on the full test set
grouped_full = predict_and_group(model_full, test_df)
results_full = eval_ranking(grouped_full)

print(f"✅ Full model results: {results_full}")

# Function to compare Top-k results to full-feature model
def compare_to_full(top_result, full_result, metric='NDCG@10'):
    if full_result > 0:
        perc = 100 * (top_result / full_result)
    else:
        perc = 0.0
    print(f"✅ Top-k retains {perc:.2f}% of full {metric} performance (Top-k: {top_result:.4f}, Full: {full_result:.4f})")
    return perc

# Update results with comparisons to full model
for res in all_results:
    res['Top NDCG@10 % full'] = compare_to_full(res['Top NDCG@10'], results_full['NDCG@10'], 'NDCG@10')
    res['Top MAP % full'] = compare_to_full(res['Top MAP'], results_full['MAP'], 'MAP')
    res['Top Precision@10 % full'] = compare_to_full(res['Top Precision@10'], results_full['Precision@10'], 'Precision@10')

# Convert to DataFrame for final summary
df_results_full = pd.DataFrame(all_results)
print("\n📈 Overall results with full model comparison:")
print(df_results_full)

# Save to CSV
df_results_full.to_csv("ranking_summary_all_k_with_full.csv", index=False)


In [ ]:
"""
This cell:
- Defines a function to plot the performance of Top-k models as a percentage of the full model's performance.
- Generates a line chart showing NDCG@10, MAP, and Precision@10 percentages across different values of k.
"""

import matplotlib.pyplot as plt

def plot_topk_vs_full(df_results):
    plt.figure(figsize=(8, 5))

    plt.plot(df_results['k'], df_results['Top NDCG@10 % full'], marker='o', label='NDCG@10 % full')
    plt.plot(df_results['k'], df_results['Top MAP % full'], marker='s', label='MAP % full')
    plt.plot(df_results['k'], df_results['Top Precision@10 % full'], marker='^', label='Precision@10 % full')

    plt.axhline(y=100, color='gray', linestyle='--', linewidth=1, label='Full model = 100%')

    plt.xlabel('Number of Top-k features')
    plt.ylabel('Performance (% of full model)')
    plt.title('Top-k performance vs full model')
    plt.ylim(90, 105)  # adjust if needed
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Call the function with the final results DataFrame
plot_topk_vs_full(df_results_full)


In [ ]:
"""
This cell:
- Defines a function to compare the performance of random-k models to the corresponding top-k model for each metric.
- Updates the results with the percentage of top-k performance achieved by random-k models and the absolute difference.
- Prints the detailed comparison.
- Saves the final results including these comparisons as a CSV file.
"""

def compare_random_to_top(top_val, random_mean, random_std, metric, k):
    if top_val > 0:
        perc = 100 * (random_mean / top_val)
    else:
        perc = 0.0
    diff = random_mean - top_val
    print(f"✅ k={k} | Random-k {metric} mean: {random_mean:.4f} ± {random_std:.4f} | Top-k: {top_val:.4f} | Diff: {diff:.4f} | Random-k = {perc:.2f}% of Top-k")
    return perc, diff

# Update results with random-k vs top-k comparisons
for res in all_results:
    perc_ndcg, diff_ndcg = compare_random_to_top(
        res['Top NDCG@10'],
        res['Random NDCG@10 mean'],
        res['Random NDCG@10 std'],
        'NDCG@10',
        res['k']
    )
    perc_map, diff_map = compare_random_to_top(
        res['Top MAP'],
        res['Random MAP mean'],
        res['Random MAP std'],
        'MAP',
        res['k']
    )
    perc_prec, diff_prec = compare_random_to_top(
        res['Top Precision@10'],
        res['Random Precision@10 mean'],
        res['Random Precision@10 std'],
        'Precision@10',
        res['k']
    )

    res['Random NDCG@10 vs Top %'] = perc_ndcg
    res['Random NDCG@10 vs Top diff'] = diff_ndcg
    res['Random MAP vs Top %'] = perc_map
    res['Random MAP vs Top diff'] = diff_map
    res['Random Precision@10 vs Top %'] = perc_prec
    res['Random Precision@10 vs Top diff'] = diff_prec

# Final summary as DataFrame
df_results_final = pd.DataFrame(all_results)
print("\n📈 Overall results with Random-k vs Top-k comparison (% and diff):")
print(df_results_final)

# Save final results
df_results_final.to_csv("ranking_summary_all_k_random_vs_top_pct.csv", index=False)


In [ ]:
"""
This cell:
- Defines a function to plot the performance of Top-k models as a percentage relative to the mean performance of Random-k models for each metric.
- Generates a line chart showing NDCG@10, MAP, and Precision@10 Top-k vs Random-k percentages across different k values.
"""

import matplotlib.pyplot as plt

def plot_topk_vs_randomk(df_results):
    plt.figure(figsize=(8,5))

    plt.plot(df_results['k'], df_results['Top NDCG@10 vs Random %'], marker='o', label='NDCG@10 Top vs Random %')
    plt.plot(df_results['k'], df_results['Top MAP vs Random %'], marker='s', label='MAP Top vs Random %')
    plt.plot(df_results['k'], df_results['Top Precision@10 vs Random %'], marker='^', label='Precision@10 Top vs Random %')

    plt.axhline(y=100, color='gray', linestyle='--', linewidth=1, label='Random mean = 100%')

    plt.xlabel('Number of Top-k features')
    plt.ylabel('Top-k performance (% of random-k mean)')
    plt.title('Top-k performance vs Random-k mean')
    plt.ylim(min(df_results[['Top NDCG@10 vs Random %', 'Top MAP vs Random %', 'Top Precision@10 vs Random %']].min()) - 2,
             max(df_results[['Top NDCG@10 vs Random %', 'Top MAP vs Random %', 'Top Precision@10 vs Random %']].max()) + 2)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
plot_topk_vs_randomk(df_results_final)


## Surrogate Fidelity: Measuring Explainability

This code evaluates whether the technique used (RankingSHAP + Decision Tree surrogate) allows us to **explain the LambdaMART model's predictions**.

### How it works:
- We train a **LambdaMART model (LightGBM Ranker)** on the full feature set.
- We select the **top-12 features** using **RankingSHAP** attributions (mean over queries).
- We fit a **DecisionTreeRegressor (max_depth=3)** on the top-12 features, aiming to approximate the LambdaMART predictions.
- We compute:
  - **R² score:** How well the surrogate's predictions match the LambdaMART predictions (closer to 1 means higher fidelity).
  - **MSE:** Mean squared error between surrogate predictions and LambdaMART predictions (lower is better).

In [ ]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error
import lightgbm as lgb

# === This script trains a LambdaMART model (LightGBM ranker) on MQ2008 LETOR data,
# uses RankingSHAP attributions to select the top-k features,
# fits a surrogate DecisionTreeRegressor to mimic the ranker's predictions using only the top-k features,
# and evaluates the fidelity (R² and MSE) of the surrogate model. ===

# === Function to read a LETOR file ===
def read_letor_file(filepath):
    X, y, qids = [], [], []
    with open(filepath, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 2:
                continue
            y_val = int(parts[0])
            qid = int(parts[1].split(":")[1])
            features = []
            for item in parts[2:]:
                if item.startswith("#"):
                    break
                _, val = item.split(":")
                features.append(float(val))
            X.append(features)
            y.append(y_val)
            qids.append(qid)
    return np.array(X), np.array(y), np.array(qids)

# === Load data ===
train_X, train_y, train_qids = read_letor_file("data/MQ2008/Fold1/train.txt")
test_X, test_y, test_qids = read_letor_file("data/MQ2008/Fold1/test_1.txt")

# === Compute group sizes correctly for LightGBM ===
df_train_qid = pd.DataFrame({'qid': train_qids})
group_sizes = df_train_qid.groupby('qid').size().tolist()

# === Load RankingSHAP feature attributions and compute top-k features ===
df_shap = pd.read_csv("results/results_MQ2008/feature_attributes/rankingshap.csv")
query_cols = [col for col in df_shap.columns if col != "feature_number"]
df_shap['mean_attribution'] = df_shap[query_cols].mean(axis=1)
df_shap_sorted = df_shap.sort_values(by='mean_attribution', ascending=False).reset_index(drop=True)
sorted_feature_indices = [int(f) - 1 for f in df_shap_sorted['feature_number']]  # zero-indexed

# Select top-k
k = 12
top_k_indices = sorted_feature_indices[:k]

# === Train LambdaMART model ===
lgbm_model = lgb.LGBMRanker(objective='lambdarank', random_state=42)
lgbm_model.fit(train_X, train_y, group=group_sizes)

# === Get predictions from the original model ===
y_pred_original = lgbm_model.predict(test_X)

# === Prepare data for the surrogate model (using only top-k features) ===
test_X_topk = test_X[:, top_k_indices]

# === Train surrogate model (Decision Tree Regressor) ===
surrogate = DecisionTreeRegressor(max_depth=3, random_state=42)
surrogate.fit(test_X_topk, y_pred_original)

# === Evaluate fidelity of the surrogate model ===
y_pred_surrogate = surrogate.predict(test_X_topk)
r2 = r2_score(y_pred_original, y_pred_surrogate)
mse = mean_squared_error(y_pred_original, y_pred_surrogate)

print(f"\n=== Surrogate Fidelity ===")
print(f"R²: {r2:.4f}")
print(f"MSE: {mse:.6f}")
